# Анализ сервиса для чтения книг

## Цель исследования

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Наша компания купила крупный сервис для чтения книг по подписке. Первая **задача аналитика — проанализировать базу данных.** В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
pd.set_option('display.max_colwidth', None)

# устанавливаем параметры
db_config = {'user': 'user', # имя пользователя
'pwd': 'pwd', # пароль
'host': 'host',
'port': 8888, # порт подключения
'db': 'db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Исследование таблиц

In [2]:
table = ['books', 'authors', 'publishers', 'ratings', 'reviews']

for i in table:
    query = '''
        SELECT *
        FROM {}
        LIMIT 5
    '''.format(i)
    
    data = pd.io.sql.read_sql(query, con = engine)
    display(data)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


### Вывод

В нашем распоряжении 5 таблиц:

1. Таблица **books** содержит данные о книгах:
 * book_id — идентификатор книги;
 * author_id — идентификатор автора;
 * title — название книги;
 * num_pages — количество страниц;
 * publication_date — дата публикации книги;
 * publisher_id — идентификатор издателя.


2. Таблица **authors** содержит данные об авторах:
 * author_id — идентификатор автора;
 * author — имя автора.


3. Таблица **publishers** содержит данные об издательствах:
 * publisher_id — идентификатор издательства;
 * publisher — название издательства;


4. Таблица **ratings** содержит данные о пользовательских оценках книг:
 * rating_id — идентификатор оценки;
 * book_id — идентификатор книги;
 * username — имя пользователя, оставившего оценку;
 * rating — оценка книги.


5. Таблица **reviews** содержит данные о пользовательских обзорах на книги:
 * review_id — идентификатор обзора;
 * book_id — идентификатор книги;
 * username — имя пользователя, написавшего обзор;
 * text — текст обзора.

## Анализ базы данных

### Количество книг, вышедших после 1 января 2000 года

In [3]:
query = '''
    SELECT
        COUNT(DISTINCT book_id)
    FROM
        books
    WHERE
        publication_date >= '2000-01-01'
'''
display(pd.io.sql.read_sql(query, con = engine))

,count
0,821


#### Вывод

После 1 января 2000 года была выпущена 821 книга.

### Количество обзоров и средняя оценка для каждой книги

Для каждой книги посчитаем количество обзоров и среднюю оценку.
Для этого объединим 3 таблицы (books, reviews & ratings) по ключу book_id. Затем сгруппируем и отсортируем. Рейтинг округлим до сотых.

ТОР10 по количеству обзоров:

In [4]:
query = '''
    SELECT b.book_id,
           b.title,
           COUNT(DISTINCT re.review_id) AS count_reviews,
           ROUND(AVG(ra.rating), 2) AS avg_rating
    FROM books AS b
    LEFT OUTER JOIN reviews AS re ON b.book_id = re.book_id
    LEFT OUTER JOIN ratings AS ra ON b.book_id = ra.book_id
    GROUP BY b.book_id
    ORDER BY count_reviews DESC, avg_rating DESC
    LIMIT 10;
'''
display(pd.io.sql.read_sql(query, con = engine))

,book_id,title,count_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,302,Harry Potter and the Prisoner of Azkaban (Harry Potter #3),6,4.41
2,299,Harry Potter and the Chamber of Secrets (Harry Potter #2),6,4.29
3,656,The Book Thief,6,4.26
4,734,The Glass Castle,6,4.21
5,497,Outlander (Outlander #1),6,4.13
6,750,The Hobbit or There and Back Again,6,4.13
7,695,The Curious Incident of the Dog in the Night-Time,6,4.08
8,779,The Lightning Thief (Percy Jackson and the Olympians #1),6,4.08
9,963,Water for Elephants,6,3.98


ТОР10 по рейтингу:

In [5]:
query = '''
    SELECT b.book_id,
           b.title,
           COUNT(DISTINCT re.review_id) AS count_reviews,
           ROUND(AVG(ra.rating), 2) AS avg_rating
    FROM books AS b
    LEFT OUTER JOIN reviews AS re ON b.book_id = re.book_id
    LEFT OUTER JOIN ratings AS ra ON b.book_id = ra.book_id
    GROUP BY b.book_id
    ORDER BY avg_rating DESC, count_reviews DESC
    LIMIT 10;
'''
display(pd.io.sql.read_sql(query, con = engine))

,book_id,title,count_reviews,avg_rating
0,17,A Dirty Job (Grim Reaper #1),4,5.0
1,553,School's Out—Forever (Maximum Ride #2),3,5.0
2,444,Moneyball: The Art of Winning an Unfair Game,3,5.0
3,169,Crucial Conversations: Tools for Talking When Stakes Are High,2,5.0
4,136,Captivating: Unveiling the Mystery of a Woman's Soul,2,5.0
5,62,Alas Babylon,2,5.0
6,57,Act of Treason (Mitch Rapp #9),2,5.0
7,20,A Fistful of Charms (The Hollows #4),2,5.0
8,55,A Woman of Substance (Emma Harte Saga #1),2,5.0
9,76,Angels Fall,2,5.0


#### Вывод

**Обзоры (TOP10):**
* Больше всех обзоров у книги: Сумерки #1 (Twilight #1)
* В ТОР3 обзоров входят сразу 2 книги о Гарри Поттере

**Рейтинг (TOP10):**
* Первое место занимает A Dirty Job (Grim Reaper #1).

### Издательство, которое выпустило наибольшее число книг толще 50 страниц

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц (чтобы исключить из анализа брошюры).
Для этого объединим 2 таблицы (publishers & books) по ключу publisher_id. Оставим только те книги, где колличество страниц более 50, сгруппируем по издательству и отсортируем по колличеству книг. Выводим только одну строку.

In [6]:
query = '''
    SELECT p.publisher,
           COUNT(b.book_id) AS count_books
    FROM publishers AS p
    LEFT OUTER JOIN books AS b ON p.publisher_id = b.publisher_id
    GROUP BY p.publisher
    ORDER BY count_books DESC
    LIMIT 1;
'''
display(pd.io.sql.read_sql(query, con = engine))

,publisher,count_books
0,Penguin Books,42


#### Вывод

Penguin Books выпустило наибольшее число книг толще 50 страниц (42 названия).

### Автор с самой высокой средней оценкой книг

Определим автора с самой высокой средней оценкой книг. Будем учитывать только книги с 50 и более оценками.

In [7]:
query = '''

SELECT                                                    
     SUBQ.author AS author,                                
     AVG(SUBQ.avg_rating) AS avg_rating
FROM
    (SELECT
         authors.author AS author,
         books.book_id AS id,
         AVG(ratings.rating) AS avg_rating,
         COUNT(ratings.rating) AS cnt_rating
    FROM
        authors
    INNER JOIN books ON books.author_id = authors.author_id
    INNER JOIN ratings ON ratings.book_id = books.book_id
    GROUP BY
        author,
        id
        HAVING COUNT(ratings.rating) >= 50) AS SUBQ
GROUP BY
    author
ORDER BY
    avg_rating DESC
LIMIT 1;     
'''
display(pd.io.sql.read_sql(query, con = engine))

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


#### Вывод

J.K. Rowling/Mary GrandPré является автором с самой высокой средней оценкой книг (среди книг с 50 и более оценками).

### Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [8]:
query = '''
    SELECT ROUND(COUNT(review_id) * 1. / COUNT(DISTINCT re.username), 2) AS avg_reviews
    FROM (
        SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(rating_id) > 50
) AS a
LEFT JOIN reviews AS re ON a.username = re.username;
'''
display(pd.io.sql.read_sql(query, con = engine))

,avg_reviews
0,24.33


#### Вывод

Среднее количество обзоров от пользователей, которые поставили больше 50 оценок равняется 24,33.

### Выводы

Основной задачей было — проанализировать базу данных сервиса для чтения книг по подписке.

В нашем распоряжении 5 таблиц:

Таблица books содержит данные о книгах.

Таблица authors содержит данные об авторах.

Таблица publishers содержит данные об издательствах.

Таблица ratings содержит данные о пользовательских оценках книг.

Таблица reviews содержит данные о пользовательских обзорах на книги.

После 1 января 2000 года была выпущена 821 книга.

**Обзоры (TOP10):**
* Больше всех обзоров у книги: Сумерки #1 (Twilight #1)
* В ТОР3 обзоров входят сразу 2 книги о Гарри Поттере

**Рейтинг (TOP10):**
* Первое место занимает A Dirty Job (Grim Reaper #1).

Penguin Books выпустило наибольшее число книг толще 50 страниц (42 названия).

J.K. Rowling/Mary GrandPré является автором с самой высокой средней оценкой книг (среди книг с 50 и более оценками).

Среднее количество обзоров от пользователей, которые поставили больше 50 оценок равняется 24,33.